In [2]:
import nltk
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("requirements.csv") # Repalce with your file path
data['requirements'] = data['feature']+ ", " + data['benefit'] + '.'
df = pd.DataFrame(list(zip(data['requirements'], data['application_domain'])),columns = ['requirements','class'])
# d['n_class'] = d['class']
# # replacing values
# d['n_class'].replace(['Health', 'Energy', 'Entertainment', 'Safety', 'Other'],[0,1,2,3,4], inplace=True)
# labels = d['n_class']
# namelabels = data['application_domain']

# merged_label = []
# for r in labels:
#     if r == 4:
#         merged_label.append(0)
#     else:
#         merged_label.append(r)


In [ ]:
print(data.to_markdown())

In [17]:
data.to_csv("requirements_merged.csv", index=False)
df.to_csv("requirements_merged_cleaned.csv", index=False)

In [ ]:
# Count the number of requirements per domain
domain_counts = data['application_domain'].value_counts().reset_index()
domain_counts.columns = ['Domain', 'Number of Requirements']
domain_counts


In [ ]:
import pandas as pd
import os

# Load the CSV
file_path = "" # Replace with your file path
df = pd.read_csv(file_path)

# Define clustering groups and sample sizes
binary_clusters = [
    ('Energy', 'Entertainment'), ('Energy', 'Safety'), ('Entertainment', 'Safety'),
    ('Health', 'Entertainment'), ('Health', 'Safety'), ('Health', 'Energy'),
    ('Energy', 'Other'), ('Entertainment', 'Other'), ('Safety', 'Other'), ('Health', 'Other')
]
tertiary_clusters = [
    ('Energy', 'Entertainment', 'Safety'), ('Health', 'Entertainment', 'Safety'),
    ('Health', 'Energy', 'Entertainment'), ('Health', 'Energy', 'Safety'),
    ('Energy', 'Entertainment', 'Other'), ('Energy', 'Safety', 'Other'),
    ('Entertainment', 'Safety', 'Other'), ('Health', 'Safety', 'Other'),
    ('Health', 'Energy', 'Other'), ('Health', 'Entertainment', 'Other')
]
quaternary_clusters = [
    ('Health', 'Energy', 'Entertainment', 'Safety'),
    ('Energy', 'Entertainment', 'Safety', 'Other'),
    ('Health', 'Energy', 'Safety', 'Other'),
    ('Health', 'Entertainment', 'Safety', 'Other'),
    ('Health', 'Energy', 'Entertainment', 'Other')
]
quinary_cluster = ['Health', 'Energy', 'Entertainment', 'Safety', 'Other']

# Sample sizes per domain for each clustering level
n_binary = 150
n_tertiary = 100
n_quaternary = 75
n_quinary = 60

# Directory to save datasets
output_dir = "" # Replace with your desired output directory
os.makedirs(output_dir, exist_ok=True)

# Sampling function allowing overlap across clusters
def sample_cluster(domains, n_per_domain, random_state=42):
    sampled_parts = []
    for domain in domains:
        df_domain = df[df['application_domain'] == domain]
        # If available less than required, sample with replacement; else without replacement
        replace = len(df_domain) < n_per_domain
        sampled = df_domain.sample(n=n_per_domain, replace=replace, random_state=random_state)
        sampled_parts.append(sampled)
    # Combine samples from different domains
    combined = pd.concat(sampled_parts).reset_index(drop=True)
    return combined

# Save helper
def save_datasets(datasets, cluster_type, domain_groups):
    saved_paths = []
    for domains, dataset in zip(domain_groups, datasets):
        filename = f"{cluster_type}_cluster_{'_'.join(domains)}.csv"
        path = os.path.join(output_dir, filename)
        dataset.to_csv(path, index=False)
        saved_paths.append(path)
    return saved_paths

# Generate and save binary clusters
binary_datasets = [sample_cluster(pair, n_binary) for pair in binary_clusters]
binary_paths = save_datasets(binary_datasets, "binary", binary_clusters)

# Generate and save tertiary clusters
tertiary_datasets = [sample_cluster(triplet, n_tertiary) for triplet in tertiary_clusters]
tertiary_paths = save_datasets(tertiary_datasets, "tertiary", tertiary_clusters)

# Generate and save quaternary clusters
quaternary_datasets = [sample_cluster(quad, n_quaternary) for quad in quaternary_clusters]
quaternary_paths = save_datasets(quaternary_datasets, "quaternary", quaternary_clusters)

# Generate and save quinary cluster
quinary_dataset = sample_cluster(quinary_cluster, n_quinary)
quinary_path = os.path.join(output_dir, "quinary_cluster_health_energy_entertainment_safety_other.csv")
quinary_dataset.to_csv(quinary_path, index=False)

# Return sample saved paths
binary_paths[:2] + tertiary_paths[:2] + quaternary_paths[:2] + [quinary_path]

